In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torchvision.transforms as T
import torchvision.datasets as dset
from torch.utils.data import DataLoader, ConcatDataset
import random
import numpy as np

In [2]:
# Hyper Parameter
NUM_EPOCHS = 20
BATCH_SIZE = 16
LEARNING_RATE = 1e-4

PRINT_EVERY = 100

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
FOLDERPATH = 'Colab\ Notebooks/Poster/a.after_test'

In [5]:
%cd drive/MyDrive/$FOLDERPATH

/content/drive/.shortcut-targets-by-id/1OC3Immm4L7H1Rp4Po9pQSFc3-pDwQpi4/Poster/a.after_test


In [6]:
# Check device
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(f'Device: {device}')

Device: cuda


In [7]:
# Fix the random seed to compare the result
def set_seed(seed=42, loader=None):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  try:
    loader.sampler.generator.manual_seed(seed)
  except AttributeError:
    pass

set_seed()

In [8]:
# Data pre-processing
transform_turing = T.Compose([
    T.Resize((300,300)),
    T.RandomHorizontalFlip(p=0.5),  # 水平翻轉
    T.RandomAutocontrast(),  # 自動調整圖像的對比度
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 幫助模型學習到不同顏色和光照條件下的圖像特徵 # 隨機調整亮度、對比度、飽和度、色調
    T.RandomRotation(10),  # 隨機將圖像在[-10度, 10度]範圍內進行旋轉
    T.RandomCrop((224,224)),
    T.ToTensor()
])
transform_new = T.Compose([
    T.Resize((300,300)),
    T.RandomCrop((224,224)),
    T.ToTensor()
])
transform_test = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

# transform = T.Compose([T.Resize((SIZE,SIZE)),
#                        T.RandomHorizontalFlip(p=0.5), # 水平翻轉
#                        T.RandomAutocontrast(), # 自動調整圖像的對比度
#                        T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # 幫助模型學習到不同顏色和光照條件下的圖像特徵 # 隨機調整亮度、對比度、飽和度、色調
#                        T.RandomRotation(10), # 隨機將圖像在[-10度, 10度]範圍內進行旋轉
#                        #T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # 標準化圖像
#                        T.ToTensor()
#                        ])

In [9]:
# Load train data
# 0: portrait  1: SD
train_data_1 = dset.ImageFolder('../Dataset/2_classes/train', transform=transform_turing)
train_data_2 = dset.ImageFolder('../new_test', transform=transform_new)
train_data_3 = dset.ImageFolder('../new_test', transform=transform_turing)
train_data_4 = dset.ImageFolder('train', transform=transform_new)
train_data_5 = dset.ImageFolder('train', transform=transform_turing)
train_data = ConcatDataset([train_data_1, train_data_2, train_data_3, train_data_4, train_data_5])

In [10]:
# Load val and test Data
# 0: portrait  1: SD
val_data = dset.ImageFolder('../Dataset/2_classes/val', transform=transform_test)
test_data = dset.ImageFolder('../Dataset/2_classes/test', transform=transform_test)

In [11]:
# Check data dimension
print(train_data[0][0].shape)
print(val_data[0][1])

torch.Size([3, 224, 224])
0


In [12]:
NUM_TRAIN = len(train_data)
NUM_VAL = len(val_data)
NUM_TEST = len(test_data)
print('Number of training:', NUM_TRAIN)
print('Number of validation:', NUM_VAL)
print('Numver of test:', NUM_TEST)

Number of training: 1856
Number of validation: 80
Numver of test: 80


In [13]:
# Build model
resnet = models.resnet50(pretrained=True)
num_flatten = resnet.fc.in_features
resnet.fc = nn.Linear(num_flatten, 2)
model = resnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 191MB/s]


In [14]:
# Set seed for DataLoader
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [15]:
# Create Mini-Batch
mini_trains = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
mini_vals = DataLoader(val_data, batch_size=BATCH_SIZE, worker_init_fn=seed_worker, generator=g)
mini_tests = DataLoader(test_data, batch_size=BATCH_SIZE)

In [16]:
# Define loss function & optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [17]:
# Training procedure
def train(mini_trains, model, NUM_EPOCHES, device):
  for epoch in range(NUM_EPOCHES):
    for count, (x, y) in enumerate(mini_trains):
      # Turn on training mode
      model.train()
      # Move data to device
      x = x.to(device)
      y = y.to(device)
      # ForwardProp
      scores = model(x)
      # Calculate loss
      loss = loss_function(scores, y)
      if count % PRINT_EVERY == 0:
        print(f'[Epoch {epoch+1}] Training loss', loss.item())
        validation(mini_vals, epoch, model, device)
      # Clear the previous gradients
      optimizer.zero_grad()
      # Get the gradients
      loss.backward()
      # Update the weights
      optimizer.step()
    print('-'*50)

In [18]:
# Validation Procedure
def validation(mini_vals, epoch, model, device):
  # Turn on val mode
  model.eval()
  with torch.no_grad():
    acc_acount = 0
    for x, y in mini_vals:
      # move data to device
      x = x.to(device)
      y = y.to(device)
      # Forward Prop
      scores = model(x)
      # Calculate accuracy
      predictions = scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_acount += acc
    print(f'[Epoch {epoch+1}] Accuracy:', acc_acount/NUM_VAL)

In [19]:
# Training
train(mini_trains, model, NUM_EPOCHS, device)

[Epoch 1] Training loss 0.7457127571105957
[Epoch 1] Accuracy: 0.4875
[Epoch 1] Training loss 0.5569329261779785
[Epoch 1] Accuracy: 0.8625
--------------------------------------------------
[Epoch 2] Training loss 0.12106165289878845
[Epoch 2] Accuracy: 0.825
[Epoch 2] Training loss 0.5025217533111572
[Epoch 2] Accuracy: 0.775
--------------------------------------------------
[Epoch 3] Training loss 0.23894865810871124
[Epoch 3] Accuracy: 0.8375
[Epoch 3] Training loss 0.28773123025894165
[Epoch 3] Accuracy: 0.8875
--------------------------------------------------
[Epoch 4] Training loss 0.047701552510261536
[Epoch 4] Accuracy: 0.9125
[Epoch 4] Training loss 0.21162129938602448
[Epoch 4] Accuracy: 0.8375
--------------------------------------------------
[Epoch 5] Training loss 0.0364924855530262
[Epoch 5] Accuracy: 0.8
[Epoch 5] Training loss 0.13468298316001892
[Epoch 5] Accuracy: 0.775
--------------------------------------------------
[Epoch 6] Training loss 0.25999510288238525


In [20]:
# Test score procedure
def test(mini_tests, model, device):
  # Use val mode to test
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_tests:
      # move data to device
      x = x.to(device)
      y = y.to(device)
      # Last Forward Prop
      score_test = model(x)
      # Calculate Accuracy
      predictions = score_test.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'[Final] Test Accuracy:', acc_count/NUM_TEST)

In [21]:
test(mini_tests, model, device)

[Final] Test Accuracy: 0.85


In [22]:
# save trained weights
torch.save(resnet.state_dict(), 'weights/resnet50_weights_mirrorup_0930.pth')

In [23]:
# # Load model and trained weights
# PATH = 'model_0907_weights.pth'

# model = models.resnet50(pretrained=True)
# num_flatten = model.fc.in_features
# model.fc = nn.Linear(num_flatten, 4)

# model.load_state_dict(torch.load(PATH))
# model = model.to(device)

In [24]:
transform_test = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

In [25]:
# Load milestone test data
new_test_data = dset.ImageFolder('../milestone', transform=transform_test)
NUM_NEW_TEST = len(new_test_data)
print('Number of new test:', NUM_NEW_TEST)
mini_tests = DataLoader(new_test_data, batch_size=1)

Number of new test: 400


In [26]:
new_test_data[0][0].shape

torch.Size([3, 224, 224])

In [27]:
# Test score procedure
def milestone_test(mini_tests, model, device):
    # Use eval mode to test
    model.eval()
    with torch.no_grad():
        acc_count = 0
        for x, y in mini_tests:
            # move data to device
            x = x.to(device)
            y = y.to(device)

            # Last Forward Prop
            score_test = model(x)

            # Calculate predictions
            predictions = score_test.max(1)[1]
            acc = predictions.eq(y).sum().item()
            acc_count += acc
        print(f'[Final] Test Accuracy:', acc_count/NUM_NEW_TEST)

In [28]:
milestone_test(mini_tests, model, device)

[Final] Test Accuracy: 0.8225
